In [1]:
import numpy as np
import sys
import os
import pandas as pd
from sklearn.metrics import ndcg_score
sys.path.append(os.path.abspath('../src'))
from model_training import get_recommendations, get_content_recommendations_pca, get_sentiment_recommendations, get_recommendations_with_pca, get_review_recommendations, get_topic_recommendations, get_reviewer_overlap_recommendations, get_weighted_hybrid_recommendations, load_all_models, get_content_recommendations, svd_product_recommendations, get_knn_recommendations, prepare_features_for_knn
# Load the DataFrame and set it in the ml module
from data_preprocessing import main
#main()
#df = pd.read_parquet('../data/processed/amazon_clean.parquet').reset_index(drop=True)
df = pd.read_parquet('../data/processed/amazon_clean.parquet')
#print(df.head())
load_all_models()

Recommendation function 'get_recommendations' updated to use reduced feature set.
All models and data loaded successfully.


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:463: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.5.1 when using version 1.8.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:463: InconsistentVersionWarning: Trying to unpickle estimator FunctionTransformer from version 1.5.1 when using version 1.8.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:463: InconsistentVersionWarning: Trying to unpickle estimator ColumnTransformer from version 1.5.1 when using version 1.8.0. This might lead to breaking

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,399.0,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,199.0,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,199.0,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,329.0,₹699,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,154.0,₹399,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1346,B08L7J3T31,Noir Aqua - 5pcs PP Spun Filter + 1 Spanner | ...,Home&Kitchen|Kitchen&HomeAppliances|WaterPurif...,379.0,₹919,59%,4.0,"1,090",SUPREME QUALITY 90 GRAM 3 LAYER THIK PP SPUN F...,"AHITFY6AHALOFOHOZEOC6XBP4FEA,AFRABBODZJZQB6Z4U...","Prabha ds,Raghuram bk,Real Deal,Amazon Custome...","R3G3XFHPBFF0E8,R3C0BZCD32EIGW,R2EBVBCN9QPD9R,R...","Received the product without spanner,Excellent...","I received product without spanner,Excellent p...",https://m.media-amazon.com/images/I/41fDdRtjfx...,https://www.amazon.in/Noir-Aqua-Spanner-Purifi...
1347,B01M6453MB,Prestige Delight PRWO Electric Rice Cooker (1 ...,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...,2280.0,"₹3,045",25%,4.1,"4,118","230 Volts, 400 watts, 1 Year","AFG5FM3NEMOL6BNFRV2NK5FNJCHQ,AGEINTRN6Z563RMLH...","Manu Bhai,Naveenpittu,Evatira Sangma,JAGANNADH...","R3DDL2UPKQ2CK9,R2SYYU1OATVIU5,R1VM993161IYRW,R...","ok,everything was good couldn't return bcoz I ...","ok,got everything as mentioned but 

In [2]:
def generate_category_ground_truth(df):
    """
    Creates a mapping of product_id -> list of other product_ids 
    in the same exact category.
    """
    ground_truth = {}
    # Group by the category string
    category_groups = df.groupby('category')['product_id'].apply(list)
    
    for category, products in category_groups.items():
        for i, target_id in enumerate(products):
            # All other products in this category are 'relevant'
            relevant = [p for j, p in enumerate(products) if i != j]
            if relevant:
                ground_truth[target_id] = relevant
                
    return ground_truth

In [3]:
import time
def evaluate_recommender(test_data, get_rec_func, K=5):
    """
    Updated: No longer passes 'df' to the recommendation function.
    """
    precisions, reciprocal_ranks, ndcgs, latencies = [], [], [], []

    for target_id, relevant_ids in test_data.items():
        # Measure latency
        start = time.time()
        # FIX: Only pass product_id and N
        recs = get_rec_func(target_id, N=K) 
        latency = time.time() - start
        latencies.append(latency)

        if hasattr(recs, 'product_id'):
            recommended_ids = recs['product_id'].tolist()
        elif isinstance(recs, list):
            recommended_ids = [r['product_id'] for r in recs if 'product_id' in r]
        else:
            recommended_ids = []

        # Precision@K
        hits = len(set(recommended_ids) & set(relevant_ids))
        precisions.append(hits / K)

        # MRR (Mean Reciprocal Rank)
        rank = 0
        for i, rid in enumerate(recommended_ids):
            if rid in relevant_ids:
                rank = i + 1
                break
        reciprocal_ranks.append(1 / rank if rank > 0 else 0)

        # NDCG calculation
        y_true = np.array([[1 if rid in relevant_ids else 0 for rid in recommended_ids]])
        y_score = np.array([[K - i for i in range(len(recommended_ids))]])
        try:
            if np.sum(y_true) > 0:
                ndcgs.append(ndcg_score(y_true, y_score, k=K))
            else:
                ndcgs.append(0.0)
        except:
            ndcgs.append(0.0)

    return {
        f"Precision@{K}": np.mean(precisions),
        "MRR": np.mean(reciprocal_ranks),
        f"NDCG@{K}": np.mean(ndcgs),
        "Latency (s)": np.mean(latencies)
    }

In [4]:
import time
# Ensure KNN features are ready
df, X_knn = prepare_features_for_knn(df)

test_set = generate_category_ground_truth(df)
# Updated test_methods to match the (pid, N) signature exactly
test_methods = {
    "Basic Cosine": lambda pid, N: get_recommendations(df, pid, N=N),
    "PCA Features": lambda pid, N: get_recommendations_with_pca(df, pid, N=N),
    "Content (TF-IDF)": lambda pid, N: get_content_recommendations(df, pid, N=N),
    "Content + PCA": lambda pid, N: get_content_recommendations_pca(df, pid, N=N),
    "Review Text": lambda pid, N: get_review_recommendations(df, pid, N=N),
    "Sentiment": lambda pid, N: get_sentiment_recommendations(df, pid, N=N),
    "Topic Modeling": lambda pid, N: get_topic_recommendations(df, pid, N=N),
    "Reviewer Overlap": lambda pid, N: get_reviewer_overlap_recommendations(df, pid, N=N),
    # For these two, we pass the global df and X_knn inside the lambda
    "SVD (Collab)": lambda pid, N: svd_product_recommendations(df, pid, n=N),
    "KNN (Numeric)": lambda pid, N: get_knn_recommendations(df, X_knn, df[df['product_id']==pid].index[0], n=N)
   # "OPTIMIZED HYBRID": lambda pid, N: get_weighted_hybrid_recommendations(pid, N=N, weights=optimal_weights)
}

# Run the evaluation
results = []
test_set_filtered = {k: v for k, v in test_set.items() if len(v) > 1}
eval_sample = dict(list(test_set_filtered.items())[:10])
print(f"Sampled {len(eval_sample)} queries for evaluation.")

#eval_sample = dict(list(hybrid_truth.items())[:30]) 

for name, func in test_methods.items():
    print(f"Evaluating {name}...", end=" ")
    try:
        metrics = evaluate_recommender(eval_sample, func, K=5)
        metrics['Model'] = name
        results.append(metrics)
        print("Done.")
    except Exception as e:
        print(f"Failed: {e}")

# Display results
kpi_df = pd.DataFrame(results)
kpi_df = kpi_df[['Model', 'NDCG@5', 'MRR', 'Precision@5', 'Latency (s)']]
display(kpi_df.sort_values(by='NDCG@5', ascending=False))

Sampled 10 queries for evaluation.
Evaluating Basic Cosine... Done.
Evaluating PCA Features... Done.
Evaluating Content (TF-IDF)... Done.
Evaluating Content + PCA... Done.
Evaluating Review Text... Done.
Evaluating Sentiment... Done.
Evaluating Topic Modeling... Done.
Evaluating Reviewer Overlap... Done.
Evaluating SVD (Collab)... Done.
Evaluating KNN (Numeric)... Done.


,Model,NDCG@5,MRR,Precision@5,Latency (s)
5,Sentiment,0.960408,0.950000,0.90,0.000807
0,Basic Cosine,0.706022,0.550000,0.38,0.001765
3,Content + PCA,0.703025,0.550000,0.36,0.001560
2,Content (TF-IDF),0.699085,0.550000,0.34,0.001768
1,PCA Features,0.683776,0.650000,0.42,0.001618
4,Review Text,0.666886,0.600000,0.38,0.001624
6,Topic Modeling,0.647691,0.583333,0.30,0.001594
8,SVD (Collab),0.608500,0.550000,0.34,0.582801
7,Reviewer Overlap,0.500000,0.600000,0.30,0.001579
9,KNN (Numeric),0.488198,0.375000,0.26,0.001391
